In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.4.0-py2.py3-none-any.whl size=350809 sha256=df547ea7e2a3d7e95eb77f34f432cc103a8cd6085736ebc440cb4b461d396b58
  Stored in directory: /root/.cache/pip/wheels/79/29/1c/234cae4632803c2ba4a76a71a679eb1383cf590775714e2a21
Successfully built emoji


In [5]:
import pandas as pd
import numpy as np
import os
import csv
import pandas as pd
import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import string
import re
import ast
import emoji 
df = pd.read_csv('/content/drive/MyDrive/PFA/DATASET/final_dataset.csv')
df = df.rename(columns={'Unnamed: 0': 'ID'})

def add_features(df):
    df['nb_posts'] = [len(c) for c in df['Dates']]
    
    df['len_caption'] = [[len(x) for x in c] for c in df['Captions']]
    
    df['mean_len'] = df['len_caption'].apply(lambda x: np.mean(x) if len(x) > 0 else 0)

    df['nb_hashtags'] = df['Captions'].apply(lambda x: sum([caption.count('#') for caption in x]))

    df['mean_likes'] = df['Likes'].apply(lambda x: np.mean(x) if len(x) > 0 else 0)

    df['mean_comments'] = df['Comments'].apply(lambda x: np.mean(x) if len(x) > 0 else 0)

def log_users_with_zero_post(df):
    is_empty = lambda x: len(x) == 0

    df['empty_posts'] = df['Dates'].apply(is_empty)
    users= df.loc[df['empty_posts'], 'user_name'].tolist()
    print("Number of users with empty list of posts: ",len(users))
    print("those folders are empty : ")
    
    if len(users) > 0:
        print("The following users have an empty list of posts: \n")
        print("\n".join(users))
        
#Clean emojis from text
def strip_emoji(text):
  """
  Remove emojis from a string.
  """
  emoji_pattern = emoji_pattern = re.compile("["
                      u"\U0001F600-\U0001F64F"  # emoticons
                      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                      u"\U0001F680-\U0001F6FF"  # transport & map symbols
                      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                      u"\U00002702-\U000027B0"
                      u"\U000024C2-\U0001F251"
                      "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', text)
#Remove punctuations, links, mentions and \r\n new line characters
def strip_all_entities(text): 
    if isinstance(text, str):
        text = text.replace('\r', '').replace('\n', ' ').lower()
        text = re.sub(r"(?:\@|https?\://)\S+", "", text)
        text = re.sub(r'[^\x00-\x7f]', r'', text)
        banned_list = string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
        table = str.maketrans('', '', banned_list)
        text = text.translate(table)
    return text
#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(caption):
    if isinstance(caption, str):
        new_caption = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', caption))
        new_caption2 = " ".join(word.strip() for word in re.split('#|_', new_caption))
        return new_caption2
    return caption

#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    if isinstance(text, str):
        return re.sub("\s\s+", " ", text)
    return text

def clean(df):
    texts_new = []
    for captions in df["Captions"]:
        cleaned_captions = []
        for caption in captions:
            clean_caption = strip_emoji(caption)
            clean_caption = strip_all_entities(clean_caption)
            clean_caption = clean_hashtags(clean_caption)
            clean_caption = filter_chars(clean_caption)
            clean_caption = remove_mult_spaces(clean_caption)
            cleaned_captions.append(clean_caption)
        texts_new.append(cleaned_captions)
    df['text_clean'] = texts_new
  
def encoding (df , att):
  df[att] = df[att].map({1.0:1,0.0:0})
  
log_users_with_zero_post(df)
df['Dates'] = df['Dates'].apply(ast.literal_eval)
df['Captions'] = df['Captions'].apply(ast.literal_eval)
df['Medias'] = df['Medias'].apply(ast.literal_eval)
df['Likes'] = df['Likes'].apply(ast.literal_eval)
df['Comments'] = df['Comments'].apply(ast.literal_eval)
add_features(df)
clean(df)
encoding(df,'I-E')
encoding(df,'S-N')
encoding(df,'T-F')
encoding(df,'J-P')
df.to_csv("/content/drive/MyDrive/PFA/DATASET/preprocessed_dataset.csv")

Number of users with empty list of posts:  0
those folders are empty : 
